In [1]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
# if packages from internet required for inference
# if iskaggle:
#    !pip install -q datasets
#    pip download datasets --dest frozen_packages --prefer-binary

In [3]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import os
from sklearn.model_selection import KFold, StratifiedKFold
import shutil
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoConfig, AutoTokenizer, get_linear_schedule_with_warmup, TrainingArguments, Trainer, AutoModelForSequenceClassification
from transformers import BertModel
from tqdm import tqdm
import warnings, transformers, logging, torch

/home/bhavik/projects/kaggle-patent-phrase-matching/venv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.environ["WANDB_DISABLED"] = "true"
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [5]:
class CFG:
    if iskaggle:
        input_path = '../input/us-patent-phrase-to-phrase-matching'
    else:
        input_path = '/home/bhavik/projects/kaggle-patent-phrase-matching/data'
    
    model_path = 'anferico/bert-for-patents'
    
    learning_rate = 1e-5
    weight_decay = 0.01
    num_fold = 5
    epochs = 5
    batch_size = 16
    max_len=133

In [6]:
train_df = pd.read_csv(f"{CFG.input_path}/train.csv")
if iskaggle:
    titles = pd.read_csv(f"../input/us-patents-category-titles/titles.csv")
else:
    titles = pd.read_csv(f"{CFG.input_path}/titles.csv")
train_df = train_df.merge(titles, left_on='context', right_on='code')

# https://www.kaggle.com/code/abhishek/phrase-matching-folds
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["fold"] = -1
    
    # the next step is to randomize the rows of the data
    # data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    # num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["score"], bins=5, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'fold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

In [7]:
def prep_input_tokens(df):
    return df.anchor + '[SEP]' + df.target + '[cpc]' + df.title

In [8]:
train_df['input'] = prep_input_tokens(train_df)
train_df = create_folds(train_df, CFG.num_fold)

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    CFG.model_path,
    additional_special_tokens = ['[abstract]','[claim]','[summary]','[invention]','[cpc]']
)

In [10]:
# ====================================================
# Define max_len
# ====================================================
lenghts = []
for input_text in train_df['input']:
    lenghts.append(len(tokenizer(input_text, add_special_tokens=False)['input_ids']))
    
max_len = max(lenghts) + 4 # CLS + SEP + SEP + SEP
CFG.max_len = max_len
max_len

70

In [11]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        self.targets = df['target'].values.astype(str)
        self.label = df['score'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        label = self.label[item]
        
        return {
        **tokenizer(inputs),
        #**tokenizer(inputs, max_length=CFG.max_len, padding="max_length"),
        #**tokenizer(inputs),
        'label': label.astype(np.float32)
    }

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [13]:
oof_df = pd.DataFrame()
for fold in range(CFG.num_fold):
    
    tr_data = train_df[train_df['fold']!=fold].reset_index(drop=True)
    va_data = train_df[train_df['fold']==fold].reset_index(drop=True)
    tr_dataset = TrainDataset(tr_data)
    va_dataset = TrainDataset(va_data)

    if iskaggle:
        train_output_dir = f"/kaggle/working/tmp/uspppm"
    else:
        train_output_dir = f"/tmp/uspppm"
    
    args = TrainingArguments(
        output_dir=train_output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=CFG.learning_rate,
        lr_scheduler_type='cosine',
        per_device_train_batch_size=CFG.batch_size,
        per_device_eval_batch_size=CFG.batch_size*2,
        num_train_epochs=CFG.epochs,
        weight_decay=CFG.weight_decay,
        metric_for_best_model="pearson",
        load_best_model_at_end=True,
    )
    
    model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
    trainer = Trainer(
        model,
        args,
        train_dataset=tr_dataset,
        eval_dataset=va_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    trainer.train()
    trainer.save_model(f"uspppm_{fold}")

    val_outputs = trainer.predict(va_dataset)
    val_predictions = val_outputs.predictions.reshape(-1)
    va_data['preds'] = val_predictions
    va_data['preds'][va_data['preds'] < 0] = 0.0
    oof_df = pd.concat([oof_df, va_data])

    shutil.rmtree(train_output_dir)

  5%|▌         | 501/9120 [01:41<27:34,  5.21it/s]

{'loss': 0.0866, 'learning_rate': 9.926019681921196e-06, 'epoch': 0.27}


 11%|█         | 1001/9120 [03:20<25:47,  5.25it/s]

{'loss': 0.0508, 'learning_rate': 9.706267962669999e-06, 'epoch': 0.55}


 16%|█▋        | 1501/9120 [05:00<25:33,  4.97it/s]

...

                                                   
100%|██████████| 9120/9120 [29:48<00:00,  5.94it/s]

{'eval_loss': 0.018755391240119934, 'eval_pearson': 0.8543612973073892, 'eval_runtime': 12.6457, 'eval_samples_per_second': 576.798, 'eval_steps_per_second': 18.03, 'epoch': 5.0}


100%|██████████| 9120/9120 [29:52<00:00,  5.09it/s]


{'train_runtime': 1792.7076, 'train_samples_per_second': 81.382, 'train_steps_per_second': 5.087, 'train_loss': 0.02133643121311539, 'epoch': 5.0}


100%|██████████| 228/228 [00:12<00:00, 13.84it/s]

In [14]:
predictions = oof_df['preds'].values
label = oof_df['score'].values
eval_pred = predictions, label
compute_metrics(eval_pred)

{'pearson': 0.8566102166369711}

In [15]:
oof_df.to_csv('oof_df_submission3.csv', index=False)

100%|██████████| 228/228 [00:26<00:00, 13.84it/s]